<p hidden>$
\newcommand{\phm}{\phantom{-}}
\newcommand{\vb}{\underline{\mathbf{b}}}
\newcommand{\vf}{\underline{\mathbf{f}}}
\newcommand{\vk}{\underline{\mathbf{k}}}
\newcommand{\vx}{\underline{\mathbf{x}}}
\newcommand{\vy}{\underline{\mathbf{y}}}
\newcommand{\deriv}[3][]{\frac{\mathrm{d}^{#1}#2}{\mathrm{d}#3^{#1}}}
\newcommand{\partderiv}[3][]{\frac{\partial^{#1}#2}{\partial#3^{#1}}}
\newcommand{\intd}{\,\mathrm{d}}
\newcommand{\rmd}{\mathrm{d}}
\DeclareMathOperator{\Uniform}{Uniform}
\DeclareMathOperator{\Poisson}{Poisson}
\DeclareMathOperator{\Normal}{Normal}
\DeclareMathOperator{\Exponential}{Exponential}
\DeclareMathOperator{\GammaDist}{Gamma}
\DeclareMathOperator{\Prob}{P}
\DeclareMathOperator{\Exp}{E}
\DeclareMathOperator{\Var}{Var}
$</p>

# Lab 4: Iterative Methods for Linear Systems

### Topics

- **Mathematics:** solving $3\times 3$ linear systems by Jacobi and Gauss–Seidel methods.
- **Python:** writing code for iterative methods for solving linear systems; using flags to indicate convergence.

In [1]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped

%matplotlib notebook
import matplotlib.pyplot as plt


## Jacobi method

Consider the linear system $A\vx=\vb$ where

$$
  A = \begin{bmatrix}
        4 & 1 & 1 \\
        1 & 2 & 3 \\
        2 & -1 & -3
      \end{bmatrix}
  \qquad
  \vb = \begin{bmatrix}
          5 \\
          5 \\
          -3
        \end{bmatrix}.
$$

Start by writing down the system of linear equations **on paper**.  Rearrange the equations into a suitable form for the Jacobi method.

Write a function `jacobi` to carry out one step of the Jacobi method for a given initial estimate $\vx^{\left(0\right)}$.

```
def jacobi(A, b, x):
    ...
    return xnew
```

Test your function using an initial estimate of $\vx^{\left(0\right)}=(0,0,0)$, checking the output with a hand calculation if necessary.

In [2]:
A = np.array([[4, 1, 1], [1, 2, 3], [2, -1, -3]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
def jacobi(A, b, x):
    xnew = np.zeros(len(x))
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    for j in range(len(x)):
        xnew[j] = (1/D[j, j])*(-sum([L[j, i]*x[i] for i in range(j-1)]) - 
                                    sum([U[j, i]*x[i] for i in range(j+1, len(x))]) + b[j])
    return xnew
    
jacobi(A, b, x)

array([1.25, 2.5 , 1.  ])

Modify your function so that:
- It carries out a series of iterations of the Jacobi method instead of just one iteration.
- It stops once it has either carried out the maximum number of iterations `niter`, or the iterations have converged within a specified tolerance `tol`, i.e. the **relative change** between successive iterations $\vx^{\left(k\right)}$ and $\vx^{\left(k+1\right)}$ is less than `tol` (just like in Lab 3):

$$
      \frac{\left\| \vx^{\left(k+1\right)}-\vx^{\left(k\right)}\right\|_\infty}
           {\left\|\vx^{\left(k+1\right)}\right\|_\infty}
        < \mathtt{tol}.
$$

- The maximum number of iterations and the acceptable tolerance are passed as inputs to the function.
- It returns a **matrix** of the results, the $k$th row of which contains $k$ and $\vx^{\left(k\right)}$, the $k$th approximation to the solution.

**Hint:** Refer to your multidimensional Newton's method function (Lab 3) as a model, particularly for the results table and convergence test. You function should look something like

```
def jacobi(A, b, x, niter, tol):
    ...
    return results
```

Try your function out using different values of `niter` and `tol` to make sure it is doing what it should do.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 1</h3>

  Print out the table returned by your function for <code>niter = 100</code> and <code>tol = 1e-6</code>.  Check that the solution from your program is correct.

  <b>Hint:</b> The results of the first 3 iterations after $\vx^{\left(0\right)}=(0,0,0)$ should be:
  <pre><code>
[[0.      1.25    2.5     1.     ]
 [1.      0.375   0.375   1.     ]
 [2.      0.90625 0.8125  1.125  ]]]
</code></pre>
</div>

In [3]:
A = np.array([[4, 1, 1], [1, 2, 3], [2, -1, -3]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
def jacobi(A, b, x, niter, tol):
    results = []
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    xnew = np.zeros(len(x))

    for k in range(niter):
        for j in range(len(x)):
            xnew[j] = (1/D[j, j])*(-sum([L[j, i]*x[i] for i in range(0, j)]) - 
                                        sum([U[j, i]*x[i] for i in range(j+1, len(x))]) + b[j])
        
        results.append([k, *xnew])
        if np.linalg.norm(xnew - x, np.inf)/np.linalg.norm(xnew, np.inf) < tol:
            break
        x = xnew.copy()
    return np.array(results)

jacobi(A, b, x, 100, 1e-6)


array([[ 0.00000000e+00,  1.25000000e+00,  2.50000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  3.75000000e-01,  3.75000000e-01,  1.00000000e+00],
       [ 2.00000000e+00,  9.06250000e-01,  8.12500000e-01,  1.12500000e+00],
       [ 3.00000000e+00,  7.65625000e-01,  3.59375000e-01,  1.33333333e+00],
       [ 4.00000000e+00,  8.26822917e-01,  1.17187500e-01,  1.39062500e+00],
       [ 5.00000000e+00,  8.73046875e-01,  6.51041667e-04,  1.51215278e+00],
       [ 6.00000000e+00,  8.71799045e-01, -2.04752604e-01,  1.58181424e+00],
       [ 7.00000000e+00,  9.05734592e-01, -3.08620877e-01,  1.64945023e+00],
       [ 8.00000000e+00,  9.14792661e-01, -4.27042643e-01,  1.70669669e+00],
       [ 9.00000000e+00,  9.30086489e-01, -5.17441361e-01,  1.75220932e+00],
       [ 1.00000000e+01,  9.41308010e-01, -5.93357227e-01,  1.79253811e+00],
       [ 1.10000000e+01,  9.50204779e-01, -6.59461174e-01,  1.82532442e+00],
       [ 1.20000000e+01,  9.58534190e-01, -7.13089013e-01,  1.85329024e+00],

Now modify your function so that it also tells you whether or not convergence has occurred:
```
def jacobi(A, b, x, niter, tol):
    ...
    return results, converged
```
True/false information like this is usually conveyed using “flag” variables containing boolean values.  In this case:

- `converged = True` if the method converged in the specified number of iterations;
- `converged = False` if the method failed to converge in the specified number of iterations.


In [4]:
def jacobi(A, b, x, niter, tol):
    results = []
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)

    for k in range(niter):
        xnew = np.zeros(len(x))
        for j in range(len(x)):
            xnew[j] = (1/D[j, j])*(-sum([L[j, i]*x[i] for i in range(0, j)]) - 
                                        sum([U[j, i]*x[i] for i in range(j+1, len(x))]) + b[j])
        
        results.append([k, *xnew])
        if np.linalg.norm(xnew - x, np.inf)/np.linalg.norm(xnew, np.inf) < tol:
            return np.array(results), True
        x = xnew
    return np.array(results), False

Now write code that uses `jacobi` to solve the system of equations above, checks the value of the output flag, and displays an appropriate message, e.g.

```
"Iterative method converged to x = ... in ... iterations"
```
or
```
"Iterative method failed to converge in ... iterations"
```

where `...` should be replaced based on your function outputs `results` and `converged`.

There are a few different ways to do string interpolation in Python, such `str.format()`,

```
print('Iterative method converged to x = {} in {} iterations'.format(x, k + 1))
```

or f-string notation,

```
print(f'Iterative method converged to x = {x} in {k + 1} iterations')
```

You can format native Python types such as floats or integers using either method, see [Fancier Output Formatting](https://docs.python.org/3/tutorial/inputoutput.html#fancier-output-formatting) for an overview and [Format String Syntax](https://docs.python.org/3/library/string.html#format-string-syntax) for more detail. You can also simplify the object that you want to insert into the string **before** it is actually inserted,

```
print('Iterative method converged to x = {} in {} iterations'.format(x.round(4), k))
```

where `x` is rounded using `np.ndarray.round`.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 2</h3>

  Run your code twice, using different values of <code>niter</code> and <code>tol</code>, once where convergence is reached and once with convergence not happening.
</div>

In [12]:
A = np.array([[4, 1, 1], [1, 2, 3], [2, -1, -3]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
def jacobi(A, b, x, niter, tol):
    results = []
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)

    for k in range(niter):
        xnew = np.zeros(len(x))
        for j in range(len(x)):
            xnew[j] = (1/D[j, j])*(-sum([L[j, i]*x[i] for i in range(0, j)]) - 
                                        sum([U[j, i]*x[i] for i in range(j+1, len(x))]) + b[j])
        
        results.append([k, *xnew])
        if np.linalg.norm(xnew - x, np.inf)/np.linalg.norm(xnew, np.inf) < tol:
            return np.array(results), True
        x = xnew
    return np.array(results), False

def print_jacobi_convergence(A, b, x, niter, tol):
    results, convergence = jacobi(A, b, x, niter, tol)
    x = results[-1][1:]
    k = results[-1][0]
    if convergence:
        print(f"Iterative method converged to x = {x.round(4)} in {k+1} iterations")
    else:
        print(f"Iterative Method failed to converge in {niter} iterations")
    
print_jacobi_convergence(A, b, x, 100, 1e-6)


Iterative method converged to x = [ 1. -1.  2.] in 72.0 iterations


In [6]:
A = np.array([[4, 1, 1], [1, 2, 3], [2, -1, -3]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
print_jacobi_convergence(A, b, x, 200, 1e-16)

Iterative Method failed to converge in 200 iterations


## Gauss–Seidel method

Write a function `gauss_seidel` that does the same thing as `jacobi`, but using the Gauss–Seidel method instead of the Jacobi method (you can use `jacobi` as a model so you don't have to start from scratch).

**Hint:** The first 3 iterations after $\vx^{\left(0\right)}=(0,0,0)$ should be:
```
[[0.         1.25       1.875      1.20833333]
 [1.         0.47916667 0.44791667 1.17013889]
 [2.         0.84548611 0.32204861 1.45630787]]
```
Write code that uses `gauss_seidel` to solve the system of equations above, checks the value of the output flag, and displays an appropriate message.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 3</h3>

  Run your code twice, using the same values of <code>niter</code> and <code>tol</code> that you used for the previous checkpoint.  Compare the Jacobi and Gauss–Seidel methods for this linear system; which is better?
</div>

In [7]:
def gauss_seidel(A, b, x, niter, tol):
    results = []
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    xnew = np.zeros(len(x))

    for k in range(niter):
        xnew = np.zeros(len(x))
        for j in range(len(x)):
            xnew[j] = (1/D[j, j])*(-sum([L[j, i]*xnew[i] for i in range(0, j)]) - 
                                        sum([U[j, i]*x[i] for i in range(j+1, len(x))]) + b[j])
        
        results.append([k, *xnew])
        if np.linalg.norm(xnew - x, np.inf)/np.linalg.norm(xnew, np.inf) < tol:
            return np.array(results), True
        x = xnew.copy()
    return np.array(results), False

def print_gauss_seidel_convergence(A, b, x, niter, tol):
    results, convergence = gauss_seidel(A, b, x, niter, tol)
    x = results[-1][1:]
    k = results[-1][0]
    if convergence:
        print(f"Iterative method converged to x = {x.round(4)} in {k} iterations")
    else:
        print(f"Iterative Method failed to converge in {niter} iterations")

        
A = np.array([[4, 1, 1], [1, 2, 3], [2, -1, -3]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
print(np.linalg.solve(A, b))

[ 1. -1.  2.]


In [8]:
print("Jacobi:")
print_jacobi_convergence(A, b, x, 100, 1e-6)
print_jacobi_convergence(A, b, x, 200, 1e-20)

Jacobi:
Iterative method converged to x = [ 1. -1.  2.] in 71.0 iterations
Iterative Method failed to converge in 200 iterations


In [9]:
print("Gauss Seidel:")
print_gauss_seidel_convergence(A, b, x, 100, 1e-6)
print_gauss_seidel_convergence(A, b, x, 200, 1e-20)


Gauss Seidel:
Iterative method converged to x = [ 1. -1.  2.] in 45.0 iterations
Iterative method converged to x = [ 1. -1.  2.] in 126.0 iterations


Now write code that uses your modified `gauss_seidel` to solve the following systems of equations.

$$
  \textrm{(i)} \qquad A = \begin{bmatrix} 4 & -2 & \phm1 \\ 3 & -6 & 1 \\ -4 & 1 & 6 \end{bmatrix} \qquad
  \vb = \begin{bmatrix}
          5 \\
          5 \\
          -3
        \end{bmatrix},
  \qquad \qquad
  \textrm{(ii)} \qquad A = \begin{bmatrix} 1 & -2 & \phm1 \\ 3 & 1 & 1 \\ -4 & 1 & 1\end{bmatrix} \qquad
  \vb = \begin{bmatrix}
          5 \\
          5 \\
          -3
        \end{bmatrix}.
$$

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 4</h3>

  What happened and why? Check your solutions are correct using <code>np.linalg.solve</code>.
</div>

In [18]:
def will_converge(A):
    for i in range(len(A)):
        if not abs(A[i, i]) > sum(abs(A[,) + sumA[i, i+1:]):
            return False
    return True



x = np.array([0, 0, 0])
A = np.array([[4, -2, 1], [3, -6, 1], [-4, 1, 6]])
print(will_converge(A))
b = np.array([5, 5, -3])
print_gauss_seidel_convergence(A, b, x, 100, 1e-6)
print(np.linalg.solve(A, b))

TypeError: 'key' is an invalid keyword argument for sum()

In [14]:
A = np.array([[1, -2, 1], [3, 1, 1], [-4, 1, 1]])
b = np.array([5, 5, -3])
x = np.array([0, 0, 0])
print(gauss_seidel(A, b, x, 100, 1e-6))
print_gauss_seidel_convergence(A, b, x, 200, 1e-6)
print(np.linalg.solve(A, b))
# A is not diagonally dominant

(array([[ 0.00000000e+000,  5.00000000e+000, -1.00000000e+001,  2.70000000e+001],
       [ 1.00000000e+000, -4.20000000e+001,  1.04000000e+002, -2.75000000e+002],
       [ 2.00000000e+000,  4.88000000e+002, -1.18400000e+003,  3.13300000e+003],
       [ 3.00000000e+000, -5.49600000e+003,  1.33600000e+004, -3.53470000e+004],
       [ 4.00000000e+000,  6.20720000e+004, -1.50864000e+005,  3.99149000e+005],
       [ 5.00000000e+000, -7.00872000e+005,  1.70347200e+006, -4.50696300e+006],
       [ 6.00000000e+000,  7.91391200e+006, -1.92347680e+007,  5.08904130e+007],
       [ 7.00000000e+000, -8.93599440e+007,  2.17189424e+008, -5.74629203e+008],
       [ 8.00000000e+000,  1.00900806e+009, -2.45239496e+009,  6.48842718e+009],
       [ 9.00000000e+000, -1.13932171e+010,  2.76912241e+010, -7.32640925e+010],
       [ 1.00000000e+001,  1.28646541e+011, -3.12675530e+011,  8.27261693e+011],
       [ 1.10000000e+001, -1.45261275e+012,  3.53057656e+012, -9.34102757e+012],
       [ 1.20000000e+001,  